In [ ]:
!pip install autots
!pip install dask
!pip install scikit-learn
!pip install FB Prophet
!pip install statsmodels
!pip install pmdarima
!pip install XGBoost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.2/830.2 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for FB: filename=fb-0.4.0-py3-none-any.whl size=7140 sha256=3a5837e34dd54dbbc0f84b8f237a4ad232ae60a532d529dfd1c971db19ea81ba
  Stored in directory: /root/.cache/pip/wheels/49/cd/23/e665d689c81d1485aa0d2aac72e7a27bbb28f0b99694c5554b
Successfully built FB
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd

datapath = '/content/dataset - dataset.csv'
dataset = pd.read_csv(datapath, engine='python')
print(dataset.shape)


(7539, 28)


In [ ]:
x = dataset[dataset['Fire_start_date'] == '2021-12-09']
x.index

Index([495], dtype='int64')

In [ ]:
dataset.iloc[495:]

,fire_year,fire_number,current_size,size_class,fire_location_latitude,fire_location_longitude,fire_origin,general_cause_desc,activity_class,det_agent,...,temperature,relative_humidity,wind_direction,wind_speed,fuel_type,initial_action_by,bh_hectares,uc_hectares,ex_hectares,Fire_start_date
495,2021,SWF167,0.15,B,55.093133,-115.952050,Provincial Land,Forest Industry,Debris Disposal,UIND,...,-15.0,65,CLM,0,C2,Industry,0.10,0.10,0.15,2021-12-09
496,2021,WWF001,0.02,A,54.272050,-116.017750,Provincial Land,Forest Industry,Debris Disposal,310,...,-3.0,90,CLM,0,M1,Industry,0.02,0.02,0.02,2021-01-08
497,2021,WWF002,0.01,A,54.759917,-114.903300,Provincial Land,Recreation,Cooking and Warming,310,...,-1.0,59,CLM,0,M1,FPD Staff,0.01,0.01,0.01,2021-01-08
498,2021,WWF003,0.20,B,54.759917,-114.903300,Provincial Land,Government,Restart,310,...,3.0,60,W,20,M1,FPD Staff,0.20,0.20,0.20,2021-01-11
499,2021,WWF004,0.04,A,54.431350,-117.260300,Provincial Land,Government,Debris Disposal,310,...,15.0,22,E,2,M1,Industry,0.04,0.04,0.04,2021-03-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7560,2006,WWF103,0.01,A,54.423050,-114.483033,Private Land,Power Line Industry,Power Lines,PUB,...,28.0,30,W,15,O1b,Fire Department,0.01,0.01,0.01,2006-08-26
7561,2006,WWF105,0.01,A,54.484876,-116.795075,Provincial Land,Oil & Gas Industry,Heavy Equipment,LFS,...,16.0,55,SE,11,S1,HAC1H,0.01,0.01,0.01,2006-09-04
7562,2006,WWF111,0.80,B,54.068434,-115.668980,Private Land,Recreation,All Terrain Vehicles,GOVT,...,13.0,35,SW,5,O1b,Fire Department,1.00,1.00,0.80,2006-10-04
7563,2006,WWF112,0.10,A,54.651933,-115.460117,Provincial Land,Power Line Industry,Power Lines,UIND,...,11.0,45,SW,12,O1a,Industry,0.10,0.10,0.10,2006-10-05


In [ ]:
#Predicting Location Through ARIMA

import pandas as pd
from datetime import datetime, timedelta
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


import pandas as pd
from datetime import datetime, timedelta
from statsmodels.tsa.arima.model import ARIMA

def predict_location(df, datetime_column, latitude_column, longitude_column):
    df_copy = df.copy()

    # Preprocessing
    df_copy[datetime_column] = pd.to_datetime(df_copy[datetime_column])
    df_copy.set_index(datetime_column, inplace=True)
    df_copy.sort_index(inplace=True)
    print(df_copy.index)
    print(list(df_copy.index))

    # Extracting latitude and longitude
    df_location = df_copy[[latitude_column, longitude_column]]

    # ARIMA model for latitude
    model_lat = ARIMA(df_location[latitude_column], order=(1, 1, 0))
    model_fit_lat = model_lat.fit()

    # ARIMA model for longitude
    model_lon = ARIMA(df_location[longitude_column], order=(1, 1, 0))
    model_fit_lon = model_lon.fit()

    # Generating future timestamps
    last_date = df_copy.index[-1].replace(hour=0, minute=0, second=0, microsecond=0)
    print(last_date)
    future_dates = [last_date + timedelta(days=i) for i in range(1, 6)]

    # Forecasting latitude and longitude
    forecast_lat = model_fit_lat.forecast(steps=5)
    forecast_lon = model_fit_lon.forecast(steps=5)

    # Creating dataframe for future dates and forecasted values
    future_df = pd.DataFrame({'DateTime': future_dates, 'forecast_lat': forecast_lat, 'forecast_lon': forecast_lon})
    future_df.set_index('DateTime', inplace=True)

    return future_df

future_df = predict_location(dataset, 'Fire_start_date', 'fire_location_latitude', 'fire_location_longitude'  )

DatetimeIndex(['2000-05-02', '2000-05-24', '2004-01-07', '2004-04-20',
               '2005-04-14', '2005-04-30', '2005-05-13', '2005-05-22',
               '2005-11-15', '2005-12-01',
               ...
               '2021-11-03', '2021-11-03', '2021-11-06', '2021-11-10',
               '2021-11-13', '2021-11-13', '2021-11-13', '2021-11-24',
               '2021-12-01', '2021-12-09'],
              dtype='datetime64[ns]', name='Fire_start_date', length=7539, freq=None)
[Timestamp('2000-05-02 00:00:00'), Timestamp('2000-05-24 00:00:00'), Timestamp('2004-01-07 00:00:00'), Timestamp('2004-04-20 00:00:00'), Timestamp('2005-04-14 00:00:00'), Timestamp('2005-04-30 00:00:00'), Timestamp('2005-05-13 00:00:00'), Timestamp('2005-05-22 00:00:00'), Timestamp('2005-11-15 00:00:00'), Timestamp('2005-12-01 00:00:00'), Timestamp('2005-12-31 00:00:00'), Timestamp('2006-01-15 00:00:00'), Timestamp('2006-01-15 00:00:00'), Timestamp('2006-01-15 00:00:00'), Timestamp('2006-01-15 00:00:00'), Timestamp('20

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


2021-12-09 00:00:00


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [ ]:
future_df

,forecast_lat,forecast_lon
DateTime,,
2021-12-10,53.251905,-115.402627
2021-12-11,54.109218,-115.659024
2021-12-12,53.710035,-115.539372
2021-12-13,53.895903,-115.595210
2021-12-14,53.809359,-115.569152


In [ ]:
future_df.to_csv('/content/drive/MyDrive/predictions.csv')